In [51]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt


In [52]:
#help(unit_square)
netgen_mesh = unit_square.GenerateMesh(maxh=0.5)
#print(netgen_mesh.MacroElementNr(...))

mesh = Mesh(netgen_mesh)
#help(mesh)
#issues with this-> maskes certain spaces not work
# mesh.Refine()
# mesh.Refine()
# mesh.Refine()
#other spaces dont work

#might need to be number of faces (in unrefined mesh) because elements are have 1 face
unref_dofnr = mesh.ne #mesh.ne
print(unref_dofnr)
Draw(mesh)

6


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [53]:


mesh.ngmesh.SplitPowellSabin()
mesh.ngmesh.Update()

#mesh._updateBuffers()


H_h = Compress(L2(mesh,order=0))
 



#might need to be number of faces in barycentrically refined mesh
bary_dofnr = H_h.ndof #mesh.ne
print(bary_dofnr)

#mesh._updateBuffers()


Draw(mesh)

36


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [54]:
#H_h = H1(mesh,order=1)
#help(H_h.Prolongation())

In [55]:
# for el in mesh.Elements():
#     print(el.nr)
    #print(len(H_h.GetDofNrs(el)))
    #print(mesh.GetParentElement(el))
    # for v in el.vertices:
    #     print(v.nr)
    # print(el.edges)
    # print(el.faces)

for f in mesh.faces:
    print(f,f.vertices)

F0 (V0, V4, V7)
F1 (V1, V4, V5)
F2 (V2, V5, V6)
F3 (V3, V6, V7)
F4 (V4, V5, V6)
F5 (V4, V6, V7)
F6 (V0, V8, V11)
F7 (V0, V10, V11)
F8 (V1, V12, V15)
F9 (V1, V14, V15)
F10 (V2, V16, V19)
F11 (V2, V18, V19)
F12 (V3, V20, V23)
F13 (V3, V22, V23)
F14 (V4, V8, V11)
F15 (V4, V9, V11)
F16 (V4, V9, V26)
F17 (V4, V13, V15)
F18 (V4, V13, V25)
F19 (V4, V14, V15)
F20 (V4, V24, V25)
F21 (V4, V24, V26)
F22 (V5, V12, V15)
F23 (V5, V13, V15)
F24 (V5, V13, V25)
F25 (V5, V17, V19)
F26 (V5, V17, V25)
F27 (V5, V18, V19)
F28 (V6, V16, V19)
F29 (V6, V17, V19)
F30 (V6, V17, V25)
F31 (V6, V21, V23)
F32 (V6, V21, V26)
F33 (V6, V22, V23)
F34 (V6, V24, V25)
F35 (V6, V24, V26)
F36 (V7, V9, V11)
F37 (V7, V9, V26)
F38 (V7, V10, V11)
F39 (V7, V20, V23)
F40 (V7, V21, V23)
F41 (V7, V21, V26)


In [56]:
 #H_h.ndof//mesh.ne
# print(netgen_mesh.dim)
#bary_dofnr//unref_dofnr == 6 (for L2)
children = np.empty((unref_dofnr,6),dtype=np.int64)

indices = np.zeros(unref_dofnr, dtype=np.int64)

for el in mesh.Elements():
    #print(el.nr)
    parent = netgen_mesh.MacroElementNr(el.nr)
    for dofnr in H_h.GetDofNrs(el):
        children[parent,indices[parent]] = dofnr
        indices[parent] += 1
    #help(el)
    #break

print(children)


[[ 0  6  7  8  9 10]
 [ 1 11 12 13 14 15]
 [ 2 16 17 18 19 20]
 [ 3 21 22 23 24 25]
 [ 4 26 27 28 29 30]
 [ 5 31 32 33 34 35]]


In [57]:
#L2 fine to coarse mapping

data_length = mesh.ne
data = np.zeros(data_length)
row_ind = np.zeros(data_length, dtype=int)
col_ind = np.zeros(data_length, dtype=int)
shape = (mesh.ne/6,mesh.ne)

j = 0

for el in mesh.Elements():
    #print(el.nr)
    parent = netgen_mesh.MacroElementNr(el.nr)
    for dofnr in H_h.GetDofNrs(el):
        row_ind[j] = parent
        col_ind[j] = dofnr
        data[j] = 1
        j += 1
    #help(el)
    #break



ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, mesh.ne//6,mesh.ne)
print(ng_mat.ToDense())

       1       0       0       0       0       0       1       1       1       1       1       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0
       0       1       0       0       0       0       0       0       0       0       0       1       1       1       1       1       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0
       0       0       1       0       0       0       0       0       0       0       0       0       0       0       0       0       1       1       1       1       1       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0
       0       0       0       1       0       0       0       0       0       0       0       0       0       0       0       0     

In [58]:
dfu = GridFunction(H_h,multidim = unref_dofnr)
#help(ng_mat)



dfu.Set(0)
for i in range(unref_dofnr):
    #print(ng_mat.ToDense().NumPy()[:,i])
    dfu.vecs[:][i] = ng_mat.ToDense().NumPy()[i,:]


# for i in range(4):
    #print(dfu.vecs[i][:].FV().NumPy())

Draw(dfu, animate = True, min = 0, max = 0.5, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [59]:
#H_h = L2(mesh,order=0) 
 


gfu = GridFunction(H_h,multidim = unref_dofnr)



In [60]:
#help(mesh)

# for el in mesh.Elements():    
#     dofnr = H_h.GetDofNrs(el)
#     #print(H_h.GetDofNrs(el.faces[0])) #same result as above as each element only contains 1 face
#     print(dofnr)

In [61]:
print(H_h.ndof)

for i in range(unref_dofnr):
    for j in children[i,:]:
        #print(j)
        gfu.vecs[i][j] = 2.0


Draw(gfu, animate = True, min = 0, max = 2.0, deformation = False)

36


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene